In [1]:
import gc
import os
import sys
import platform
import logging
import argparse
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np

repo_path = '/home/notebook'
sys.path.append(os.path.join(repo_path, 'test'))


# from common.cpoall_aggregator import run_daily_tms_analysis
# Import necessary files and its respective functions
from common.db_operations import connect_to_trino,execute_query, fetch_distinct_ids_for_day_trino,fetch_data_for_day_trino, write_df_to_iceberg, parse_arguments, get_target_date
from common.cpoall_aggregator import run_daily_tms_analysis,CORE_COLS

In [2]:
# tms_fault_code
# tms_working_mode
# ac_operating_mode
# b2t_tms_control_cmd
# v2t_vehicle_coolant_low

In [3]:
conn = connect_to_trino()
sql = f"select * from facts_prod.can_output_ac where date(timestamp) > date('2025-12-23')"
df = execute_query(conn, sql, return_results=True)
conn.close()

2025-12-24 07:49:57 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:49:57 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:49:57 - INFO - ⚙️ Executing query...
2025-12-24 07:50:03 - INFO - ✅ Query executed successfully with results!


In [4]:
conn = connect_to_trino()
sql = f"select distinct b2t_tms_control_cmd from facts_prod.can_output_ac"
df1 = execute_query(conn, sql, return_results=True)
display(df1)
conn.close()

2025-12-24 07:50:03 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:50:03 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:50:03 - INFO - ⚙️ Executing query...
2025-12-24 07:50:22 - INFO - ✅ Query executed successfully with results!


,b2t_tms_control_cmd
0,None
1,Charging_Cooling
2,Off
3,Fast_Discharge_Cooling
4,Self_Circulation


In [5]:
conn = connect_to_trino()
sql = f"select distinct tms_working_mode from facts_prod.can_output_ac"
df1 = execute_query(conn, sql, return_results=True)
display(df1)
conn.close()

2025-12-24 07:50:22 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:50:22 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:50:22 - INFO - ⚙️ Executing query...
2025-12-24 07:50:30 - INFO - ✅ Query executed successfully with results!


,tms_working_mode
0,Fast_Discharge_Cooling
1,Off
2,Low_Coolant
3,Self_Circulation
4,Charging_Cooling
5,None


In [10]:
conn = connect_to_trino()
sql = f"select distinct tms_fault_code from facts_prod.can_output_ac"
df1 = execute_query(conn, sql, return_results=True)
display(df1)
conn.close()

2025-12-24 07:50:56 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:50:56 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:50:56 - INFO - ⚙️ Executing query...
2025-12-24 07:51:02 - INFO - ✅ Query executed successfully with results!


,tms_fault_code
0,Water_Sensor_Failure
1,LV Undervoltage
2,173
3,Low_Water_Level_Alarm
4,None
5,No Fault
6,Water_IN_Sensor_Failure
7,Water_Pump_Failure
8,Exhaust_Temp_Protection


In [7]:
conn = connect_to_trino()
sql = f"select distinct ac_operating_mode from facts_prod.can_output_ac"
df1 = execute_query(conn, sql, return_results=True)
display(df1)
conn.close()

2025-12-24 07:50:37 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:50:37 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:50:37 - INFO - ⚙️ Executing query...
2025-12-24 07:50:43 - INFO - ✅ Query executed successfully with results!


,ac_operating_mode
0,Cooling
1,Defrosting
2,Heating_Mode
3,None
4,Free
5,Air_Supply_Mode
6,Forced_Heating
7,Forced_Cooling


In [8]:
conn = connect_to_trino()
sql = f"select distinct v2t_vehicle_coolant_low from facts_prod.can_output_ac"
df1 = execute_query(conn, sql, return_results=True)
display(df1)
conn.close()

2025-12-24 07:50:43 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-24 07:50:43 - INFO - ✅ STEP 1: Connected to Trino
2025-12-24 07:50:43 - INFO - ⚙️ Executing query...
2025-12-24 07:50:50 - INFO - ✅ Query executed successfully with results!


,v2t_vehicle_coolant_low
0,None
1,Normal


In [9]:
display(df.head())

,id,date,timestamp,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,b2t_tms_control_cmd,b2t_set_water_out_temp,b2t_battery_min_temp,b2t_battery_max_temp,tms_working_mode,tms_fault_code,coolant_out_temp,coolant_in_temp,hv_voltage,comp_target_hz,comp_status,comp_running_frequency,ac_fault_code,outside_temp,cabin_temp,blower_speed,ac_status,ac_set_temp,ac_operating_mode,v2t_vehicle_coolant_low,comp_current,ac_system_ipm_module_temp,ac_system_eva_temp,dt
0,15,2025-12-24,2025-12-24 01:50:14.490,2025-12-24 01:50:34.876,20793,2,2,33.333336,Off,15.0,24.0,30.0,Off,No Fault,22.0,22.0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,2025-12-24
1,15,2025-12-24,2025-12-24 01:50:16.150,2025-12-24 01:50:39.902,22140,3,4,50.000000,Off,15.0,24.0,30.0,None,None,NaN,NaN,616.0,0.0,Off,0.0,No_Fault,NaN,NaN,NaN,None,NaN,None,Normal,NaN,NaN,NaN,2025-12-24
2,15,2025-12-24,2025-12-24 01:50:18.170,2025-12-24 01:50:39.902,23743,4,4,66.666670,Off,15.0,24.0,30.0,None,None,NaN,NaN,617.0,0.0,Off,0.0,No_Fault,NaN,NaN,NaN,None,NaN,None,Normal,0.0,26.0,23.0,2025-12-24
3,15,2025-12-24,2025-12-24 01:50:20.390,2025-12-24 01:50:44.927,25543,4,7,66.666670,Off,15.0,24.0,30.0,Off,No Fault,22.0,22.0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,Normal,0.0,26.0,23.0,2025-12-24
4,15,2025-12-24,2025-12-24 01:50:21.910,2025-12-24 01:50:44.927,26763,3,3,50.000000,Off,15.0,24.0,30.0,None,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,Normal,0.0,26.0,23.0,2025-12-24


In [2]:
CORE_COLS = [
    # Metadata & Base Info
    "id", "timestamp", "dt",

    # Commands
    "air_compressor_enable_command",

    # cell voltage/temperature
    "air_compreesor_temperature", "oilpumpcondition","airpumpcondition", "lowpressureoilpumpfaultcode", "airpumpdcaclifesignal", 

    # Electrical
    "soh", "bat_voltage", "bat_soc","total_battery_current", "insulation__value",

    # DriveMode
    "odometerreading", "gear_position", "vehicle_operation_mode",

    # Alarms & Faults (from previous list)
    "temperaturedifferencealarm", "bms_fault_code", "vcu_fault_code"
]

In [ ]:
# conn = connect_to_trino()   # your DB function
# day_str = "2025-10-29"
# core_cols = CORE_COLS
# source_schema = "facts_prod"
# source_table = "can_parsed_output_all"
# ids = ['6']

# df_raw1 = fetch_data_for_day_trino(conn=conn,day_str=day_str,ids=ids,core_cols=core_cols,table=source_table,schema=source_schema,)
# conn.close()

2025-12-26 06:18:06 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:18:06 - INFO - ✅ STEP 1: Connected to Trino


In [4]:
from datetime import date, timedelta
import pandas as pd

def daterange(start: date, end: date):
    """Yield each date from start to end, inclusive."""
    d = start
    while d <= end:
        yield d
        d += timedelta(days=1)

def fetch_data_for_range_trino(
    start_date: str,
    end_date: str,
    ids,
    core_cols,
    schema: str,
    table: str,
):
    """
    Fetch day-by-day using your existing fetch_data_for_day_trino(conn, day_str, ...),
    then concat into one DataFrame.
    """
    start = date.fromisoformat(start_date)
    end   = date.fromisoformat(end_date)

    dfs = []
    for d in daterange(start, end):
        day_str = d.isoformat()

        # Option 1 (robust): connect per day
        conn = connect_to_trino()
        try:
            df_day = fetch_data_for_day_trino(
                conn=conn,
                day_str=day_str,
                ids=ids,
                core_cols=core_cols,
                table=table,
                schema=schema,
            )
        finally:
            conn.close()

        if df_day is not None and len(df_day) > 0:
            dfs.append(df_day)

    if not dfs:
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)

In [5]:
from datetime import date, timedelta

def fetch_data_for_delta_days_trino(
    start_date: str,
    delta_days: int,
    ids,
    core_cols,
    schema: str,
    table: str,
):
    start = date.fromisoformat(start_date)
    # delta_days=1 means "just start_date"
    end = start + timedelta(days=delta_days - 1)
    return fetch_data_for_range_trino(
        start_date=start.isoformat(),
        end_date=end.isoformat(),
        ids=ids,
        core_cols=core_cols,
        schema=schema,
        table=table,
    )


# # --- Usage: start_date + end_date (inclusive) ---
# core_cols = CORE_COLS
# source_schema = "facts_prod"
# source_table = "can_parsed_output_all"
# ids = ["6"]

# df_raw = fetch_data_for_range_trino(
#     start_date="2025-10-29",
#     end_date="2025-11-02",
#     ids=ids,
#     core_cols=core_cols,
#     schema=source_schema,
#     table=source_table,
# )


# --- Usage: start_date + delta days ---
df_raw = fetch_data_for_delta_days_trino(
    start_date="2025-12-10",
    delta_days=14,  # 2025-10-29 .. 2025-11-02
    
    # "6","7": Freshbus
    # "3", "16": Zing Bus

    ids=["3","6","7","16"],
    core_cols=CORE_COLS,
    schema="facts_prod",
    table="can_parsed_output_all",
)


2025-12-26 06:31:43 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:31:43 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:31:49 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:31:49 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:31:55 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:31:55 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:01 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:32:01 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:06 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:32:06 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:12 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:32:12 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:18 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:32:18 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:24 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-26 06:32:24 - INFO - ✅ STEP 1: Connected to Trino
2025-12-26 06:32:30 - INFO - 🔌 STEP 1: C

In [7]:
df_raw.head()

,id,timestamp,dt,air_compressor_enable_command,air_compreesor_temperature,oilpumpcondition,airpumpcondition,lowpressureoilpumpfaultcode,airpumpdcaclifesignal,soh,bat_voltage,bat_soc,total_battery_current,insulation__value,odometerreading,gear_position,vehicle_operation_mode,temperaturedifferencealarm,bms_fault_code,vcu_fault_code
0,16,2025-12-09 18:30:00.380,2025-12-09,0.0,37.0,1.0,0.0,0.0,87.0,96.8,633.3,78.0,88.9,4999.0,70044.750,2.0,1.0,0.0,0.0,0.0
1,6,2025-12-09 18:30:00.453,2025-12-09,0.0,72.0,1.0,0.0,0.0,96.0,96.0,635.6,92.4,93.5,NaN,86720.625,2.0,1.0,0.0,0.0,0.0
2,7,2025-12-09 18:30:00.895,2025-12-09,0.0,51.0,1.0,0.0,0.0,251.0,95.6,NaN,NaN,NaN,4999.0,NaN,2.0,1.0,0.0,0.0,0.0
3,3,2025-12-09 18:30:01.221,2025-12-09,0.0,59.0,1.0,0.0,0.0,6.0,98.0,638.2,90.8,2.3,NaN,67143.125,2.0,1.0,0.0,0.0,0.0
4,7,2025-12-09 18:30:02.275,2025-12-09,0.0,51.0,1.0,0.0,0.0,25.0,95.6,633.2,89.2,194.0,NaN,99722.000,2.0,1.0,0.0,0.0,0.0


In [8]:
df_raw.to_csv('compressor_4bus_data.csv')